## Seminar Assignment 1
Inteligentni sistemi, 20. 11. 2022   
Klemen Bogataj in Aljaž Rakovec

Seminarsko nalogo sva delala v Pythonu, za genetski algoritem pa sva uporabila knjižnico PyGAD (https://github.com/ahmedfgad/GeneticAlgorithmPython).

#### Reprezentacija
Labirint preberemo v NumPy matriko, kjer številke predstavljajo različna polja:
- \# -> 0
- . -> 1
- S -> 2
- E -> 3
- T -> 4

Vse ostale parametre določimo ob inicializaciji genetskega algoritma.

### Fitness funkcije
- ```fitness_func_klemen```   
Iz reštve vzamemo samo prvih toliko potez, dokler ne pridemo do zidu ali cilja. Če smo dosegli cilj, je vrednost 1000 - dolžina poti, torej kaznujemo daljše poti, če cilja še nismo dosegli, pa je vrednost število obiskanih polj - dolžina poti / 1000, torej nagradimo rešitve, ki prepodujejo čimvečji del labirinta in jim to uspe s čim manj potezami.
Težava je, da ne upoštevamo poti po prvem zadetem zidu, torej ne moremo oceniti uspešnosti sprememb od tam naprej.
- ```fitness_func_klemen2```   
Tukaj za začetno populacijo generiramo rešitve, ki vse pridejo do cilja, ampak grejo čez zidove. Vzamemo poteze od začetka do prvega prispetja na cilj. Za ta del poti izračunamo število prepotovanih različnih veljavnih polj, število potez, ki končajo izven mej labirinta in število zadetih zidov. Če smo do cilja prišli po samih veljavnih poljih, je vrednost 1000000 - dolžina poti, sicer pa najbolj kaznujemo poteze izven labirinta, nato zadete zidove, nagradimo pa poti, ki obiščejo čimveč različnih veljavnih polj.
- ```fitness_func_aljaz```   
Fukncija gre iterira čez rešitev. Če rešitev pripelje do zidu ali gre izven meje labirinta, zmanjša število točk. Prav tako ga zmanjša, če se vrne na isto polje kot je bil en korak nazaj. Če pride na polje, mu prišteje število točk glede na trenutno oddaljenost od cilja. Če pride do zaklada mu prav tako poveča število točk, ampak samo enkrat na vsak zaklad. Ko pide do cilja, število točko zelo poveča in odšteje število korakov, ki jih je rešitev porabila, da je prišla do cilja. S tem maksimiziramo najkrajšo rešitev. Funkcija vrne število točk, ko pride do cilja ali pa do konca rešitve. S pravimi mutacijami in crossoverjem lahko reši največ labirint 6.

**Končna fitness funkcija:**   
Ker zdaj vse rešitve hodijo samo po veljavnih poljih, se ni traba ukvarjati z zidovi. Dokler še nismo na cilju nagradimo daljše poti ter obilno nagradimo vsak najden zaklad, ko pa prispemo na cilj, pa poleg nagrajevanja zakladov še obilneje nagradimo prispetje na cilj, dolžino poti pa tokrat kaznujemo.

```Python
def fitness_points(solution, solution_idx):
    length = 0
    found_treasures = set()
    current_point = start_point
    for i in range(solution.size):
        current_move = int(solution[i])
        length += moves_to_next_points[current_point][current_move].size
        current_point = next_points[current_point, current_move]
        if current_point == end_point:
            return 1000000 + len(found_treasures) * 10000 - length
        elif np.any(treasures == current_point):
            found_treasures.add(current_point)
            pass
    return len(found_treasures) * 10000 + length
```

### Crossover funkcija

- ```crossover_func_same_coordinate```   
Funkcija iz seznama koordinat cele poti obeh strašev in njade katere koordinate imata enake. Če nimata nobene, potem izvede vgrajeni single-point crossover, v ostalih primerih pa izvede single-point crossover na eni izmed skupnih koordinat. Funkcija ni izboljšala delovanja od uporabe vgrajenega uniform crossoverja.  
- Za vse nekončne rešitve sva uporabljala vgrajeni uniform crossover ali pa crossoverja sploh nisva uporabila.

**Končna crossover funkcija**:  
V dveh rešitvah poiščemo skupne odločitvene točke, torej tiste, ki so vsaj enkrat obiskane pri obeh starših. Naključno izberemo eno izmed njih, in ker se lahko v rešitvi pojavi večkrat, še nakljušno izberemo eno izmed pojavitev. Potomce generiramo tako, da vzamemo del poti od začetka do izbrane pojavitve od enega starša, drugi del od te točke naprej pa od drugega. tudi ostala dela združimo, tako da imamo dva nova potomca. Ker lahko pride do drugačne dolžine kromosoma, daljšega na koncu odrežemo na želeno dolžino, krajšemu pa na koncu dodamo nove poteze tako, kot smo jih generirali za začetno populacijo. Tako dosežemo, da še vedno nobena pot nikoli ne gre čez zid.

```Python
def crossover_points(parents, offspring_size, ga_instance):
    height, width =  maze.shape
    offspring = []
    id_parent = 0
    while len(offspring) != offspring_size[0]:
        # choose two parents
        parent1 = parents[id_parent % parents.shape[0], :].copy()
        parent2 = parents[(id_parent + 1) % parents.shape[0], :].copy()

        # get all decision points for both parents
        points1 = decision_points_visited(parent1)[:-1]
        points2 = decision_points_visited(parent2)[:-1]

        # calculates same points and randomly cooses one of them
        same_points = set(points1).intersection(set(points2))
        same_point = random.sample(same_points, 1)

        # chooses one of the indexes of same_point for both parents
        idx1_same_point = random.sample(set(np.where(points1 == same_point)[0]), 1)[0] + 1
        idx2_same_point = random.sample(set(np.where(points2 == same_point)[0]), 1)[0] + 1

        # splits parents in firt and second part
        if idx1_same_point < idx2_same_point:
            short_first_part = parent1[:idx1_same_point]
            long_first_part = parent2[:idx2_same_point]
            long_second_part = parent1[idx1_same_point:]
            short_second_part = parent2[idx2_same_point:]
        else:
            short_first_part = parent2[:idx2_same_point]
            long_first_part = parent1[:idx1_same_point]
            long_second_part = parent2[idx2_same_point:]
            short_second_part = parent1[idx1_same_point:]

        # size difference
        difference = abs(idx1_same_point - idx2_same_point)

        # creates and adds the first child
        long_child = np.concatenate((long_first_part, long_second_part))
        if difference > 0:
            long_child = long_child[:-difference]
        offspring.append(long_child)

        # creates and adds the second child if there is not enough offspring yet
        if (len(offspring) < offspring_size[0]):
            short_child = np.concatenate((short_first_part, short_second_part))
            short_child = extend(short_child, difference)
            offspring.append(short_child)
        
        id_parent += 1
    #print(np.array(offspring))
    return np.array(offspring)
```

### Mutation funkcija

- ```mutation_func_aljaz```   
Funkcija gre čez podani gen. Če prideme do zidu ali izven polja labirinta, naključno spremeni spremik v eno izmed ostalih treh smeri. Če je med sosednjimi kordinatami trenutne pozicije cilj, ga preusmeri tja in zaključi mutacijo. Mutacija zelo pomaga pri reševanu do sedmega labirinta. Zraven je bil uporabljen uniform crossover in verjetnost za mutacijo 100%.
- ```mutation_func_two_adjacent_genes```   
Izberemo naljučno mesto od začetka do prvega prispetja na cilj in tam izvedemo mutacijo odvisno od naslednjih dveh potez:
    - Če sta potezi v eno smer in nazaj, izbrišemo te dve potezi, vse naslednje premaknemo za dve mesti v levo in na koncu dodamo dve naključni novi.
    - Če sta potezi pravokotni, ju zamenjamo, torej na nek način obrnemo vogal, ki smo ga našli.
    - Če sta potezi obe v isto smer, gremo po eni strani okrog vmesnega polja. Torej pred njiju vrinemo eno pravokotno potezo, za njiju pa pravokotno potezo ravno v drugo stran. Vse za tem zamaknemo za dve mesti naprej, pri tem se zadnji dve izbrišeta. LL -> ULLD / DLLU
Na ta način dosežemo, da poti pred in po mutaciji gresta po istem delu labirinta kot prej in s tem preprečimo možnost, da pot ne bi več prišla do cilja.

**Končna mutation funkcija:**   
Izberemo nakjučno mesto v rešitvi in od tam naprej naključno zgeneriramo veljavne poteze, tako kot pri začetni populaciji. Tako gre nova rešitev še vedno samo po veljavnih poljih.

```Python
def mutation_points(offspring, ga_instance):
    for i in range(offspring.shape[0]):
        mutation_locaion = random.randint(0, offspring.shape[1] - 1)
        current_point = start_point
        for j in range(mutation_locaion):
            current_point = next_points[current_point, int(offspring[i, j])]
        for j in range(mutation_locaion, offspring.shape[1]):
            move = random.randint(0, 3)
            point = next_points[current_point, move]
            while point == -1:
                move = random.randint(0, 3)
                point = next_points[current_point, move]
            offspring[i, j] = move
            current_point = point
    return offspring
```

### Inicializacija začetne populacije:
```initial_to_end_through_walls```:   
Generiramo poteze, ki ne grejo nujno po veljavnih poljih, vendar približno na polovici pridejo do cilja. To dosežemo tako, da izračunamo oddaljenost starta in cilja po obeh dimenzijah, zgeneriramo ustrezno večje ali manjše število potez gor, dol ter levo, desno, nato pa te poteze naključno premešamo. Preostali del poti zapolnimo z naključnimi potezami. Cilj na sredini omogoča, da se pot do tja skrajšuje ali podaljšuje, ne da bi cilj izpadel iz poti.


**Končna inicializacija začetne populacije:**  
V večini točk je nesmiselno iti v smer zidu ali nazaj, od koder smo prišli, zato najprej v labirintu poiščemo "točke odločitve", to so start, cilj, zakladi in veljavna polja, ki imajo vsaj tri sosednja veljavna polja. To pomeni, da če gremo od ene izmed teh točk v neko smer, nadaljujemo po poti vse do naslednje take točke, ne da bi se vmes obračali ali zaletavali v zid. Zato zdaj nima več smisla v populacijo pisati vseh potez, ampak samo poteze na teh točkah, saj so povezave med njimi jasne. Začetne rešitve torej generiramo tako, da začnemo na cilju, izberemo eno imed veljavnih potez, pogledamo do katere odločitvene točke bomo prišli po tej poti, nato pa pri te in vseh nasledjih odločitvenih točkah spet naključno izberemo eno izmed možnih veljavnih potez. Tako so vse rešitve zapisane v veliko krajši obliki in vse se po labitintu sprehajajo samo po veljavnih poljih.

```Python
def population_points(maze, sol_per_pop, length_factor):
    num_points = next_points.shape[0]
    num_genes = num_points * length_factor
    population = np.zeros((sol_per_pop, num_genes), dtype=np.int32)
    for i in range(sol_per_pop):
        #population[i, 0] = start_point
        current_point = start_point
        # generates a valid sequence of decision points as one solution
        for j in range(0, num_genes):
            move = random.randint(0, 3)
            point = next_points[current_point, move]
            while point == -1:
                move = random.randint(0, 3)
                point = next_points[current_point, move]
            population[i, j] = move
            current_point = point
            
    return population
```

Primer izbire odločitvenih točk:    
<img src="odlocitvene_tocke.png" alt="Drawing" style="width: 300px;"/>

### Inicializacija genteskega algoritma

Primer instance genetskega algoritma za končno verzijo:

```Python
sol_per_pop = 50
keep_elitism = 10
num_parents_maiting = 20
length_factor = 3
population_p = population_points(maze, sol_per_pop, length_factor)

ga_instance = pygad.GA(num_generations=2000,
                       num_parents_mating=num_parents_maiting,
                       #sol_per_pop=sol_per_pop,
                       initial_population=population_p,
                       gene_space=[0, 1, 2, 3],
                       keep_elitism=keep_elitism,
                       stop_criteria="saturate_100",
                       crossover_type=crossover_points,
                       mutation_type=mutation_points,
                       fitness_func=fitness_points,
                       on_generation=on_generation,
                       on_stop=on_stop_points)
```

Zagon genetskega algoritma:

```Python
ga_instance.run()

```

#### Graf časa izvajanja glede na velikost labirinta
![Alt text](graf1.png)

#### Graf števila generacij in časa glede na velikost labirinta
![Alt text](graf2.png)